In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [30]:
df = pd.read_csv('../artifacts/eda_gemstone.csv')
df.head()
# df = pd.read_csv('./data/gemstone.csv')
# df.drop('id', axis=1, inplace=True)
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,4,3,4,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,3,7,2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,5,4,5,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,5,4,5,61.6,56.0,4.38,4.41,2.71,666
4,1.70,4,4,4,62.6,59.0,7.65,7.61,4.77,14453


In [31]:
X = df.drop(labels=['price'],axis=1)
Y = df[['price']]

In [32]:
categorical = X.select_dtypes(include='object').columns
numerical = X.select_dtypes(exclude='object').columns

In [33]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [34]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [35]:
# Numerical Pipeline

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

num_pipe = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())           
    ]
)

preprocessor=ColumnTransformer([
('num_pipeline',num_pipe,numerical),
])

In [ ]:
cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
color_map = {"D":1 ,"E":2 ,"F":3 , "G":4 ,"H":5 , "I":6, "J":7}
clarity_map = {"I1":1,"SI2":2 ,"SI1":3 ,"VS2":4 , "VS1":5 , "VVS2":6 , "VVS1":7 ,"IF":8}

In [37]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=30)

In [38]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=X_train.columns)
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=X_test.columns)

In [39]:
X_train

,carat,cut,color,clarity,depth,table,x,y,z
0,-0.543215,-0.133859,-1.549661,-1.315382,0.073369,0.402260,-0.438843,-0.482164,-0.445311
1,0.947760,-1.140082,0.299681,0.017068,0.997890,-1.160093,0.993521,1.031776,1.100588
2,-0.824123,0.872364,-0.933214,1.349518,0.258273,-0.639309,-0.862246,-0.890111,-0.853661
3,-0.824123,0.872364,1.532576,0.683293,0.073369,-0.639309,-0.871254,-0.844784,-0.839077
4,-1.040207,-1.140082,2.149024,0.683293,0.535629,0.402260,-1.240606,-1.243666,-1.203676
...,...,...,...,...,...,...,...,...,...
135482,0.277901,-0.133859,0.916129,0.683293,-0.388892,1.964613,0.543092,0.496911,0.473478
135483,-0.975382,0.872364,1.532576,1.349518,0.535629,-1.160093,-1.114486,-1.116749,-1.072421
135484,0.645243,0.872364,0.299681,2.015743,-0.019083,-1.160093,0.804341,0.841400,0.823493
135485,-1.018598,-0.133859,0.299681,0.017068,0.535629,1.443828,-1.267632,-1.243666,-1.218260


In [40]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [41]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [42]:
regression.coef_

array([[ 6495.16594914,    74.38970588,  -467.24284938,   652.52467056,
          172.67721693,   -63.6630948 ,  -392.73879184,  1352.6183836 ,
        -3306.07931642]])

In [43]:
regression.intercept_

array([3977.74781344])

In [44]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [45]:
## Train multiple models
## Model Ecaluation
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1002.5666320523034
MAE: 669.8919608762216
R2 score 93.7766128728445


Lasso
Model Training Performance
RMSE: 1003.647220595244
MAE: 671.0630291331818
R2 score 93.76319023389532


Ridge
Model Training Performance
RMSE: 1002.563344439937
MAE: 669.9262381408552
R2 score 93.77665368818839


Elasticnet
Model Training Performance
RMSE: 1524.444646489965
MAE: 1060.5733301375362
R2 score 85.61123088114182




In [18]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']